In [31]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)

In [22]:
mdf = pd.read_csv('M_athlete_results.csv')
fdf = pd.read_csv('F_athlete_results.csv')

C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
csm = mdf.loc[(mdf['School'] == 'COLORADO MINES') & (mdf['Season'] == 'Outdoor') & (mdf['Event'] == '5000')]
csm.sort_values('Mark')[['Name', 'Mark', 'Place', 'Meet_Name', 'Year']].head(93)

,Name,Mark,Place,Meet_Name,Year
82800,MARTY ANDRIE,13:44.38,11.0,Mt. SAC Relays,NaN
82819,MARTY ANDRIE,13:57.29,7.0,Mt. SAC Relays,NaN
82820,MARTY ANDRIE,14:02.81,14.0,Stanford Invitational,NaN
65642,JOSH HOSKINSON,14:04.33,14.0,Bryan Clay Invitational,NaN
65658,JOSH HOSKINSON,14:05.42,3.0,Stanford Invitational,NaN
96331,ANDREW EPPERSON,14:09.02,16.0,Stanford Invitational,NaN
64774,LUKE JULIAN,14:10.94,9.0,2019 Oxy Invitational,2
74334,MATTHEW KADE,14:11.40,6.0,Bryan Clay Invitational,NaN
45676,GRANT COLLIGAN,14:12.27,13.0,Mt. SAC Relays,4
105166,SEAN GILDEA,14:15.81,6.0,Mt. SAC Relays,NaN


In [49]:
mdf.loc[mdf['Name']=='LUKE JULIAN']

,Name,Athlete ID,Grade,Year,School,Conference,Meet_ID,Meet_Name,Meet_Start,Meet_End,Year.1,Season,Event,Mark,Place,Prelim/Final
64768,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,14511,2020 RMAC Cross Country Championships,"Oct 24, 2020","Oct 24, 2020",2020,Cross Country,8K,24:40.4,8.0,NaN
64769,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,62377,Chile Pepper XC Fesitval,"Oct 5, 2019","Oct 5, 2019",2019,Cross Country,8K,24:23.3,23.0,NaN
64770,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,58041,12th Annual UCCS Rust-Buster - 2019,"Sep 7, 2019","Sep 7, 2019",2019,Cross Country,8K,24:54.4,6.0,NaN
64771,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,52971,NCAA Division II Outdoor Track & Field Champio...,"May 23, 2019","May 25, 2019",2019,Outdoor,1500,3:55.57,9.0,P
64772,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,52971,NCAA Division II Outdoor Track & Field Champio...,"May 23, 2019","May 25, 2019",2019,Outdoor,5000,14:53.87,13.0,F
64773,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,17299,2019 Oxy Invitational,"May 11, 2019","May 11, 2019",2019,Outdoor,1500,3:44.45,2.0,F
64774,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,17299,2019 Oxy Invitational,"May 11, 2019","May 11, 2019",2019,Outdoor,5000,14:10.94,9.0,F
64775,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,61281,RMAC Outdoor Track & Field Championships,"Apr 26, 2019","Apr 28, 2019",2019,Outdoor,1500,3:53.44,1.0,P
64776,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,14554,Colorado Invitational 2019,"Apr 5, 2019","Apr 6, 2019",2019,Outdoor,1500,3:48.42,3.0,F
64777,LUKE JULIAN,6603636,SO,2,COLORADO MINES,NaN,57853,NCAA Division II Indoor Track & Field Champion...,"Mar 8, 2019","Mar 9, 2019",2019,Indoor,3000,8:14.06,5.0,F


In [48]:
mdf.loc[mdf['Name']=='CHRIS CATHCART']    # Roster page may be blank

,Name,Athlete ID,Grade,Year,School,Conference,Meet_ID,Meet_Name,Meet_Start,Meet_End,Year.1,Season,Event,Mark,Place,Prelim/Final
